In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Run this to attack a trained model via TrainModel. 
Use the "loadFullModel" submethod to load in an already trained model (trained via TrainModel)
The main attack function is "runAttacks" which runs attacks on trained models
"""
import warnings
warnings.filterwarnings("ignore")

import os
import tensorflow as tf
#tf.compat.v1.enable_eager_execution
os.environ['CUDA_VISIBLE_DEVICES'] = '0' 
from cleverhans.attacks import Noise, CarliniWagnerL2, MaxConfidence, FastGradientMethod, BasicIterativeMethod, DeepFool, MomentumIterativeMethod, ProjectedGradientDescent
from Model_Implementations import Model_Softmax_Baseline, Model_Logistic_Baseline, Model_Logistic_Ensemble, Model_Tanh_Ensemble, Model_Tanh_Baseline
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras import backend
import numpy as np
import scipy.linalg
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [ ]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
model_path = 'models/'  #path with saved model parameters 
sess =  backend.get_session()
backend.set_learning_phase(0) #need to do this to get CleverHans to work with batchnorm


#Dataset-specific parameters - should be same as those used in TrainModel

#CIFAR
# DATA_DESC = 'CIFAR10'; (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
# epochs=None; weight_save_freq=None
# num_classes=10  #how many classes (categories) are in this dataset?
# Y_train = np.squeeze(Y_train); Y_test = np.squeeze(Y_test)
# num_filters_std = [32, 64, 128]; num_filters_ens=[32, 64, 128]; num_filters_ens_2=16; dropout_rate_std=0.0; dropout_rate_ens=0.0; weight_decay = 0 
# model_rep_baseline=2; model_rep_ens=2; DATA_AUGMENTATION_FLAG=1; BATCH_NORMALIZATION_FLAG=1
# num_channels = 3; inp_shape = (32,32,3); lr=1e-4; batch_size=80;
# noise_stddev = 0.032; blend_factor = .032

#MNIST
DATA_DESC = 'MNIST'; (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
Y_train = np.squeeze(Y_train); Y_test = np.squeeze(Y_test)
num_channels = 1; inp_shape = (28,28,1); num_classes=10
#MODEL-SPECIFIC PARAMETERS: MNIST
#PARAMETERS RELATED TO SGD OPTIMIZATION
epochs=None; weight_save_freq=None; batch_size=80; lr=3e-4; 
#MODEL DEFINTION PARAMETERS
num_filters_std = [64, 64, 64]; num_filters_ens=[32, 32, 32]; num_filters_ens_2=4; 
dropout_rate_std=0.0; dropout_rate_ens=0.0; weight_decay = 0 
noise_stddev = 0.3; blend_factor=0.3; 
model_rep_baseline=1; model_rep_ens=2; 
DATA_AUGMENTATION_FLAG=0; BATCH_NORMALIZATION_FLAG=0

#Attack parameters
eps_val = 8/255.0; PGD_iters = 200; eps_iter=(2/3)*eps_val; 
eps_range = np.linspace(0, 0.33, 10)
noise_eps=0.1


# DATA PRE-PROCESSING
X_train = (X_train/255).astype(np.float32);  X_test = (X_test/255).astype(np.float32)
#reshape (add third (image) channel)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],num_channels); X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],num_channels)
X_valid = X_test[1000:2000]; Y_valid = Y_test[1000:2000]; #validation data, used to attack model
X_train = X_train-0.5; X_test = X_test-0.5; X_valid = X_valid-0.5; #map to range (-0.5,0.5)
data_dict = {'X_train':X_train, 'Y_train_cat':Y_train, 'X_test':X_test, 'Y_test_cat':Y_test}
X_random = np.random.rand(X_valid.shape[0],X_valid.shape[1],X_valid.shape[2],X_valid.shape[3])-0.5; X_random = X_random.astype(np.float32)


# def output_activation(x):
#     return tf.nn.softmax(x) 
name = 'logistic_diverse'+'_'+DATA_DESC; num_chunks=2
M = np.eye(num_classes).astype(np.float32)
base_model=None
def output_activation(x):
    return tf.nn.sigmoid(x)
params_dict = {'BATCH_NORMALIZATION_FLAG':BATCH_NORMALIZATION_FLAG, 'DATA_AUGMENTATION_FLAG':DATA_AUGMENTATION_FLAG, 'M':M, 'base_model':base_model, 'num_chunks':num_chunks, 'model_rep': model_rep_ens, 'output_activation':output_activation, 'num_filters_ens':num_filters_ens, 'num_filters_ens_2':num_filters_ens_2,'batch_size':batch_size, 'epochs':epochs, 'dropout_rate':dropout_rate_ens,  'lr':lr, 'blend_factor':blend_factor, 'inp_shape':inp_shape, 'noise_stddev':noise_stddev, 'weight_save_freq':weight_save_freq, 'name':name, 'model_path':model_path}
m4 = Model_Logistic_Ensemble(data_dict, params_dict)
m4.loadFullModel()

#Model definition of the model we want to attack; should be same as the definition used in TrainModel
# name = 'tanh_16_diverse'+'_'+DATA_DESC; seed = 59; code_length=16; num_codes=code_length; num_chunks=4; base_model=None; 
# def output_activation(x):
#     return tf.nn.tanh(x)
# M = scipy.linalg.hadamard(code_length).astype(np.float32)
# M[np.arange(0, num_codes,2), 0]= -1#replace first col, which for scipy's Hadamard construction is always 1, hence not a useful classifier; this change still ensures all codewords have dot product <=0; since our decoder ignores negative correlations anyway, this has no net effect on probability estimation
# np.random.seed(seed)
# np.random.shuffle(M)
# idx=np.random.permutation(code_length)
# M = M[0:num_codes, idx[0:code_length]]
# params_dict = {'BATCH_NORMALIZATION_FLAG':BATCH_NORMALIZATION_FLAG, 'DATA_AUGMENTATION_FLAG':DATA_AUGMENTATION_FLAG, 'M':M, 'base_model':base_model, 'num_chunks':num_chunks, 'model_rep': model_rep_ens, 'output_activation':output_activation, 'num_filters_ens':num_filters_ens, 'num_filters_ens_2':num_filters_ens_2,'batch_size':batch_size, 'epochs':epochs, 'dropout_rate':dropout_rate_ens,  'lr':lr, 'blend_factor':blend_factor, 'inp_shape':inp_shape, 'noise_stddev':noise_stddev, 'weight_save_freq':weight_save_freq, 'name':name, 'model_path':model_path}
# m4 = Model_Tanh_Ensemble(data_dict, params_dict)
# m4.loadFullModel() #load in the saved model, which should have already been trained first via TrainModel

m4.legend = 'TEns16'; 
m4.X_valid = X_valid; m4.Y_valid = Y_valid; 
m4.X_test = X_test; m4.Y_test = Y_test; 
m4.X_random = X_random; 
m4.minval = -0.5; m4.maxval = 0.5



def benignAccuracy(model, X, Y):
    
    acc_vec=[]; probs_benign_list=[]
    for rep in np.arange(0, X.shape[0], 1000):
        x = X[rep:rep+1000]
        probs_benign = sess.run(model.predict(tf.convert_to_tensor(x))) 
        acc = np.mean(np.argmax(probs_benign, 1)==Y[rep:rep+1000])
        acc_vec += [acc]
        probs_benign_list += list(np.max(probs_benign, 1))

        
    acc = np.mean(acc_vec)        
    print("Accuracy for model " + model.params_dict['name'] + " : ", acc)    
    return probs_benign_list


def wbAttack(model, attack, att_params, X, Y):
    sess =  backend.get_session()
    modelCH = model.modelCH()
    adv_model = attack(modelCH, sess=sess) 
    
    acc_vec=[]; probs_adv_list=[]
    inc=250
    for rep in np.arange(0, X.shape[0], inc):
        x = X[rep:rep+inc]
        y = Y[rep:rep+inc]
        X_adv = adv_model.generate(tf.convert_to_tensor(x), **att_params).eval(session=sess)  
        print(X_adv.dtype)
        preds = np.argmax(sess.run(model.predict(tf.convert_to_tensor(X_adv))), 1)
        acc =  np.mean(np.equal(preds, y))
        probs_adv = np.max(sess.run(model.predict(tf.convert_to_tensor(X_adv))), 1)
        probs_adv = probs_adv[preds != y]
        acc= np.mean(np.equal(preds, y))
        acc_vec += [acc]
        probs_adv_list += list(probs_adv)

        
    acc = np.mean(acc_vec)        
    print("Adv accuracy for model " + model.params_dict['name'] + " : ", acc)    
    return probs_adv_list, acc, X_adv, y


import timeit

start = timeit.default_timer()

stop = timeit.default_timer()
print('Time: ', stop - start)  


def runAttacks(models_list):
    #CW attack
    for model in models_list:
        
        print(""); print(""); print("");
        print("Running tests on model: ", model.params_dict['name'])
        start = timeit.default_timer()
        print("Clean accuracy of model:")
        probs_benign = benignAccuracy(model, model.X_test, model.Y_test)
        print("")
        stop = timeit.default_timer()
        print('Benign Time: ', stop - start)  
        
#         start = timeit.default_timer()
#         print("Running PGD attack:")
#         att_params = {'clip_min': model.minval, 'clip_max':model.maxval, 'eps':eps_val, 'eps_iter':eps_iter, 'nb_iter':PGD_iters,'ord':np.inf}
#         probs_adv_pgd, junk, X_adv_pgd, y_adv_pgd = wbAttack(model, ProjectedGradientDescent, att_params, model.X_valid, model.Y_valid)
#         print("")
#         stop = timeit.default_timer()
#         print('PGD Time: ', stop - start) 
        
#         start = timeit.default_timer()
#         print("Running CW attack:")
#         att_params = {'clip_min': model.minval, 'clip_max':model.maxval,  'binary_search_steps':10, 'learning_rate':1e-3}
#         probs_adv_cw, junk, X_adv, y = wbAttack(model, CarliniWagnerL2, att_params, model.X_valid[0:100], model.Y_valid[0:100])
#         print("")
#         stop = timeit.default_timer()
#         print('CW Time: ', stop - start) 
        
#         start = timeit.default_timer()
#         print("Running Blind Spot attack, alpha=0.8:")
#         att_params = {'clip_min': model.minval, 'clip_max':model.maxval,  'binary_search_steps':10, 'learning_rate':1e-3}
#         probs_adv_bsa, junk, X_adv, y = wbAttack(model, CarliniWagnerL2, att_params, 0.8*model.X_valid[0:100], model.Y_valid[0:100])
#         print("")
#         stop = timeit.default_timer()
#         print('BSA Time: ', stop - start) 
                
        #Random ATTACK (0 SNR inputs)
#         start = timeit.default_timer()
#         print("Running random attack:")
#         probs_random = np.max(sess.run(model.predict(tf.convert_to_tensor(model.X_random))), 1)
#         print('Prob. that ', model.params_dict['name'], ' < 0.9 on random data: ', np.mean(probs_random<0.9))
#         stop = timeit.default_timer()
#         print('Random Time: ', stop - start) 
        
#         #Noise ATTACK (low SNR inputs)
#         start = timeit.default_timer()
#         print("Running Noise attack:")
#         att_params = {'clip_min': model.minval, 'clip_max':model.maxval, 'eps':noise_eps}
#         probs_noise, junk, X_adv, y = wbAttack(model, Noise, att_params, model.X_valid, model.Y_valid)
#         print("")
#         stop = timeit.default_timer()
#         print('Running Noise Time: ', stop - start) 
        
#     return probs_benign, probs_adv_pgd, probs_adv_cw, probs_adv_bsa, probs_random, probs_noise, X_adv_pgd, y_adv_pgd
#     return probs_adv_pgd, X_adv_pgd, y_adv_pgd #for plotting probs



models_list = [m4]
# probs_benign, probs_adv_pgd, probs_adv_cw, probs_adv_bsa, probs_random, probs_noise, X_adv_pgd, y_adv_pgd = runAttacks(models_list)
# probs_adv_pgd, X_adv_pgd, y_adv_pgd = runAttacks(models_list) #for ploting probs


In [ ]:
# DAA for Logistic Ensemble models

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from scipy.spatial.distance import pdist, squareform
from matplotlib import pyplot as plt

c = 1.1
import time

class LinfBLOBAttack:
    def __init__(self, model, epsilon, k, a, random_start):
        self.model = model
        self.epsilon = epsilon
        self.k = k
        self.a = a
        self.rand = random_start

        c = 1.1
        print('c: ', c)

    def perturb(self, x_nat, x_adv, y, sess): 
        batch_size = x_adv.shape[0]
        y = tf.expand_dims(y, axis=1)
#         y = tf.expand_dims(y, axis=1)
        y = tf.dtypes.cast(y, tf.float32)
        for epoch in range(10):
#             print(epoch)
            x_adv = tf.convert_to_tensor(x_adv)
            x_adv = tf.dtypes.cast(x_adv, tf.float32)
            with tf.GradientTape(persistent=True) as g:
                g.watch(x_adv)
    #             out = tf.argmax(m4.model(x_nat),axis=1)
    #             out = tf.expand_dims(out, axis=1)
    #             out = tf.dtypes.cast(out, tf.float64)
    #             y = tf.dtypes.cast(y, tf.int64)
    #             loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=out)
                out = m4.model(x_adv)
                out = tf.dtypes.cast(out, tf.float32)
#                 loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out)
                loss = [m4.defineLoss(k)(y,out) for k in np.arange(m4.params_dict['num_chunks'])]
#                 loss = tf.reduce_mean(loss,0)

            grad = g.gradient(loss, x_adv)
            grad = grad.eval(session=sess)
            x_adv = x_adv.eval(session=sess)
            grad = np.reshape(grad, [grad.shape[0], grad.shape[1]*grad.shape[2]*grad.shape[3]])
            kxy, dxkxy = self.svgd_kernel(x_adv)
            x_adv = np.reshape(x_adv, [x_adv.shape[0], x_adv.shape[1]*x_adv.shape[2]*x_adv.shape[3]])
            x_adv += self.a * np.sign(c*(-(np.matmul(kxy, -grad) + dxkxy)/batch_size) + grad)
            
            #we need to put back the dimensions since our model has a 4-dimensional input
            x_adv = np.reshape(x_adv, [x_adv.shape[0], 28, 28, 1])
            
            x_adv = np.clip(x_adv, x_nat - self.epsilon, x_nat + self.epsilon) 
            
            x_adv = np.clip(x_adv, 0, 1) # ensure valid pixel range

        return x_adv
    
    def svgd_kernel(self, theta):
        theta = np.reshape(theta, [theta.shape[0], theta.shape[1]*theta.shape[2]])
        sq_dist = pdist(theta)
        pairwise_dists = squareform(sq_dist)**2

        h = np.median(pairwise_dists)  
        h = np.sqrt(0.5 * h / np.log(theta.shape[0]))

        # compute the rbf kernel
        Kxy = np.exp( -pairwise_dists / h**2 / 2)

        dxkxy = -np.matmul(Kxy, theta)
        sumkxy = np.sum(Kxy, axis=1)
        for i in range(theta.shape[1]):
            dxkxy[:, i] = dxkxy[:,i] + np.multiply(theta[:,i],sumkxy)
        dxkxy = dxkxy / (h**2)
        return (Kxy, dxkxy)


if __name__ == '__main__':
    import json
    import sys
    import math
    
    sess =  backend.get_session()

    attack = LinfBLOBAttack(m4,
                         epsilon=0.3,
                         k=200,
                         a=0.01,
                         random_start=True)

    num_eval_examples = 1000
    eval_batch_size = 200
    num_batches = int(math.ceil(num_eval_examples / eval_batch_size))

    print('Iterating over {} batches'.format(num_batches))

    #     x_adv_final = np.copy(mnist.test.images)

    for restart in range(50):
      # Initialize permutation
        permutation = np.arange(num_eval_examples)
        idx = np.arange(num_eval_examples)

        
        (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
        Y_test = np.squeeze(Y_test)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],num_channels)
        
        x_test = X_test 
        y_test = Y_test
        
        x_adv = x_test + np.random.uniform(-attack.epsilon, attack.epsilon, x_test.shape)
        

      # per round
        t0 = time.time()
        for epoch in range(int(attack.k/10)):
            print(epoch)
            print(int(attack.k/10))
            np.random.shuffle(idx)
            x_test, x_adv, y_test = x_test[idx], x_adv[idx], y_test[idx]
            permutation = permutation[idx]
            
            for ibatch in range(num_batches):
                bstart = ibatch * eval_batch_size
                bend = min(bstart + eval_batch_size, num_eval_examples)
                
                x_batch = x_test[bstart:bend, :]
                x_batch_adv = x_adv[bstart:bend, :]
                y_batch = y_test[bstart:bend]
            x_adv[bstart:bend, :] = attack.perturb(x_batch, x_batch_adv, y_batch, sess)
            

        inv_permutation = np.argsort(permutation)
        x_adv = x_adv[inv_permutation]
 
        
        t1 = time.time()

        print('restart: ', restart, '   time per batch: ', t1 - t0)

        print('L2: ', np.mean(np.square(x_adv - x_test)))
        print('Linf: ', np.max(np.abs(x_adv - x_test)))
        x_adv = tf.convert_to_tensor(x_adv)
        x_adv = tf.dtypes.cast(x_adv, tf.float32)
        preds = np.argmax(sess.run(attack.model.predict(x_adv)), 1)
        acc = np.mean(np.equal(preds, y_test))
        print('adv acc: ', acc)

In [ ]:
# # DAA for Tahn16 Ensemble models

# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
# from scipy.spatial.distance import pdist, squareform
# from matplotlib import pyplot as plt

# c = 1.1
# import time

# class LinfBLOBAttack:
#     def __init__(self, model, epsilon, k, a, random_start):
#         self.model = model
#         self.epsilon = epsilon
#         self.k = k
#         self.a = a
#         self.rand = random_start

#         c = 1.1
#         print('c: ', c)

#     def perturb(self, x_nat, x_adv, y, sess): 
#         batch_size = x_adv.shape[0]
#         y = tf.expand_dims(y, axis=1)
#         y = tf.dtypes.cast(y, tf.float32)
#         for epoch in range(10):
# #             print(epoch)
#             x_adv = tf.convert_to_tensor(x_adv)
#             x_adv = tf.dtypes.cast(x_adv, tf.float32)
#             with tf.GradientTape(persistent=True) as g:
#                 g.watch(x_adv)
#     #             out = tf.argmax(m4.model(x_nat),axis=1)
#     #             out = tf.expand_dims(out, axis=1)
#     #             out = tf.dtypes.cast(out, tf.float64)
#     #             y = tf.dtypes.cast(y, tf.int64)
#     #             loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=out)
#                 out = m4.model(x_adv)
#                 out = tf.dtypes.cast(out, tf.float32)
# #                 loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out)
#                 loss = [m4.defineLoss(k)(y,out) for k in np.arange(m4.params_dict['num_chunks'])]
# #                 loss = tf.reduce_mean(loss,0)

#             grad = g.gradient(loss, x_adv)
#             grad = grad.eval(session=sess)
#             x_adv = x_adv.eval(session=sess)
#             grad = np.reshape(grad, [grad.shape[0], grad.shape[1]*grad.shape[2]*grad.shape[3]])
#             kxy, dxkxy = self.svgd_kernel(x_adv)
#             x_adv = np.reshape(x_adv, [x_adv.shape[0], x_adv.shape[1]*x_adv.shape[2]*x_adv.shape[3]])
#             x_adv += self.a * np.sign(c*(-(np.matmul(kxy, -grad) + dxkxy)/batch_size) + grad)
            
#             #we need to put back the dimensions since our model has a 4-dimensional input
#             x_adv = np.reshape(x_adv, [x_adv.shape[0], 28, 28, 1])
            
#             x_adv = np.clip(x_adv, x_nat - self.epsilon, x_nat + self.epsilon) 
            
#             x_adv = np.clip(x_adv, 0, 1) # ensure valid pixel range

#         return x_adv
    
#     def svgd_kernel(self, theta):
#         theta = np.reshape(theta, [theta.shape[0], theta.shape[1]*theta.shape[2]])
#         sq_dist = pdist(theta)
#         pairwise_dists = squareform(sq_dist)**2

#         h = np.median(pairwise_dists)  
#         h = np.sqrt(0.5 * h / np.log(theta.shape[0]))

#         # compute the rbf kernel
#         Kxy = np.exp( -pairwise_dists / h**2 / 2)

#         dxkxy = -np.matmul(Kxy, theta)
#         sumkxy = np.sum(Kxy, axis=1)
#         for i in range(theta.shape[1]):
#             dxkxy[:, i] = dxkxy[:,i] + np.multiply(theta[:,i],sumkxy)
#         dxkxy = dxkxy / (h**2)
#         return (Kxy, dxkxy)


# if __name__ == '__main__':
#     import json
#     import sys
#     import math
    
#     sess =  backend.get_session()

#     attack = LinfBLOBAttack(m4,
#                          epsilon=0.3,
#                          k=200,
#                          a=0.01,
#                          random_start=True)

#     num_eval_examples = 1000
#     eval_batch_size = 200
#     num_batches = int(math.ceil(num_eval_examples / eval_batch_size))

#     print('Iterating over {} batches'.format(num_batches))

#     #     x_adv_final = np.copy(mnist.test.images)

#     for restart in range(50):
#       # Initialize permutation
#         permutation = np.arange(num_eval_examples)
#         idx = np.arange(num_eval_examples)

        
#         (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
#         Y_test = np.squeeze(Y_test)
#         X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],num_channels)
        
#         x_test = X_test 
#         y_test = Y_test
        
#         x_adv = x_test + np.random.uniform(-attack.epsilon, attack.epsilon, x_test.shape)
        

#       # per round
#         t0 = time.time()
#         for epoch in range(int(attack.k/10)):
#             print(epoch)
#             print(int(attack.k/10))
#             np.random.shuffle(idx)
#             x_test, x_adv, y_test = x_test[idx], x_adv[idx], y_test[idx]
#             permutation = permutation[idx]
            
#             for ibatch in range(num_batches):
#                 bstart = ibatch * eval_batch_size
#                 bend = min(bstart + eval_batch_size, num_eval_examples)
                
#                 x_batch = x_test[bstart:bend, :]
#                 x_batch_adv = x_adv[bstart:bend, :]
#                 y_batch = y_test[bstart:bend]
#             x_adv[bstart:bend, :] = attack.perturb(x_batch, x_batch_adv, y_batch, sess)
            

#         inv_permutation = np.argsort(permutation)
#         x_adv = x_adv[inv_permutation]
 
        
#         t1 = time.time()

#         print('restart: ', restart, '   time per batch: ', t1 - t0)

#         print('L2: ', np.mean(np.square(x_adv - x_test)))
#         print('Linf: ', np.max(np.abs(x_adv - x_test)))
#         x_adv = tf.convert_to_tensor(x_adv)
#         x_adv = tf.dtypes.cast(x_adv, tf.float32)
#         preds = np.argmax(sess.run(attack.model.predict(x_adv)), 1)
#         acc = np.mean(np.equal(preds, y_test))
#         print('adv acc: ', acc)

In [ ]:
# # DAA for Tahn16 Standard models

# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
# from scipy.spatial.distance import pdist, squareform
# from matplotlib import pyplot as plt

# c = 1.1
# import time

# class LinfBLOBAttack:
#     def __init__(self, model, epsilon, k, a, random_start):
#         self.model = model
#         self.epsilon = epsilon
#         self.k = k
#         self.a = a
#         self.rand = random_start

#         c = 1.1
#         print('c: ', c)

#     def perturb(self, x_nat, x_adv, y, sess): 
#         batch_size = x_adv.shape[0]
#         for epoch in range(10):
# #             print(epoch)
#             x_adv = tf.convert_to_tensor(x_adv)
#             x_adv = tf.dtypes.cast(x_adv, tf.float32)
#             with tf.GradientTape(persistent=True) as g:
#                 g.watch(x_adv)
#     #             out = tf.argmax(m4.model(x_nat),axis=1)
#     #             out = tf.expand_dims(out, axis=1)
#     #             out = tf.dtypes.cast(out, tf.float64)
#     #             y = tf.dtypes.cast(y, tf.int64)
#     #             loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=out)
#                 out = m4.model(x_adv)
#                 # the next 2 lines are for the sigmoid!!!!!
# #                 out = tf.math.argmax(out, 1)
# #                 out = tf.expand_dims(out, axis=1)
                
#                 y = tf.expand_dims(y, axis=1)
#                 out = tf.dtypes.cast(out, tf.float32)
#                 #for sigmoid we need float32
#                 y = tf.dtypes.cast(y, tf.float32)
# #                 y = tf.dtypes.cast(y, tf.int64)
# #                 loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out)
#                 loss = [m4.defineLoss(k)(y,out) for k in np.arange(m4.params_dict['num_chunks'])]

#             grad = g.gradient(loss, x_adv)
#             grad = grad.eval(session=sess)
#             x_adv = x_adv.eval(session=sess)
#             grad = np.reshape(grad, [grad.shape[0], grad.shape[1]*grad.shape[2]*grad.shape[3]])
#             kxy, dxkxy = self.svgd_kernel(x_adv)
#             x_adv = np.reshape(x_adv, [x_adv.shape[0], x_adv.shape[1]*x_adv.shape[2]*x_adv.shape[3]])
#             x_adv += self.a * np.sign(c*(-(np.matmul(kxy, -grad) + dxkxy)/batch_size) + grad)
            
#             #we need to put back the dimensions since our model has a 4-dimensional input
#             x_adv = np.reshape(x_adv, [x_adv.shape[0], 28, 28, 1])
            
#             x_adv = np.clip(x_adv, x_nat - self.epsilon, x_nat + self.epsilon) 
            
#             x_adv = np.clip(x_adv, 0, 1) # ensure valid pixel range

#         return x_adv
    
#     def svgd_kernel(self, theta):
#         theta = np.reshape(theta, [theta.shape[0], theta.shape[1]*theta.shape[2]])
#         sq_dist = pdist(theta)
#         pairwise_dists = squareform(sq_dist)**2

#         h = np.median(pairwise_dists)  
#         h = np.sqrt(0.5 * h / np.log(theta.shape[0]))

#         # compute the rbf kernel
#         Kxy = np.exp( -pairwise_dists / h**2 / 2)

#         dxkxy = -np.matmul(Kxy, theta)
#         sumkxy = np.sum(Kxy, axis=1)
#         for i in range(theta.shape[1]):
#             dxkxy[:, i] = dxkxy[:,i] + np.multiply(theta[:,i],sumkxy)
#         dxkxy = dxkxy / (h**2)
#         return (Kxy, dxkxy)


# if __name__ == '__main__':
#     import json
#     import sys
#     import math
    
#     sess =  backend.get_session()

#     attack = LinfBLOBAttack(m4,
#                          epsilon=0.3,
#                          k=200,
#                          a=0.01,
#                          random_start=True)

#     num_eval_examples = 1000
#     eval_batch_size = 200
#     num_batches = int(math.ceil(num_eval_examples / eval_batch_size))

#     print('Iterating over {} batches'.format(num_batches))

#     #     x_adv_final = np.copy(mnist.test.images)

#     for restart in range(50):
#       # Initialize permutation
#         permutation = np.arange(num_eval_examples)
#         idx = np.arange(num_eval_examples)

        
#         (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
#         Y_test = np.squeeze(Y_test)
#         X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],num_channels)
        
#         x_test = X_test 
#         y_test = Y_test
        
#         x_adv = x_test + np.random.uniform(-attack.epsilon, attack.epsilon, x_test.shape)
        

#       # per round
#         t0 = time.time()
#         for epoch in range(int(attack.k/10)):
#             print(epoch)
#             np.random.shuffle(idx)
#             x_test, x_adv, y_test = x_test[idx], x_adv[idx], y_test[idx]
#             permutation = permutation[idx]
            
#             for ibatch in range(num_batches):
#                 bstart = ibatch * eval_batch_size
#                 bend = min(bstart + eval_batch_size, num_eval_examples)
                
#                 x_batch = x_test[bstart:bend, :]
#                 x_batch_adv = x_adv[bstart:bend, :]
#                 y_batch = y_test[bstart:bend]
#             x_adv[bstart:bend, :] = attack.perturb(x_batch, x_batch_adv, y_batch, sess)
            

#         inv_permutation = np.argsort(permutation)
#         x_adv = x_adv[inv_permutation]
 
        
#         t1 = time.time()

#         print('restart: ', restart, '   time per batch: ', t1 - t0)

#         print('L2: ', np.mean(np.square(x_adv - x_test)))
#         print('Linf: ', np.max(np.abs(x_adv - x_test)))
#         x_adv = tf.convert_to_tensor(x_adv)
#         x_adv = tf.dtypes.cast(x_adv, tf.float32)
#         preds = np.argmax(sess.run(attack.model.predict(x_adv)), 1)
#         acc = np.mean(np.equal(preds, y_test))
#         print('adv acc: ', acc)

In [ ]:
# # DAA for Softmax Standard models

# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
# from scipy.spatial.distance import pdist, squareform
# from matplotlib import pyplot as plt

# c = 1.1
# import time

# class LinfBLOBAttack:
#     def __init__(self, model, epsilon, k, a, random_start):
#         self.model = model
#         self.epsilon = epsilon
#         self.k = k
#         self.a = a
#         self.rand = random_start

#         c = 1.1
#         print('c: ', c)

#     def perturb(self, x_nat, x_adv, y, sess): 
#         batch_size = x_adv.shape[0]
#         for epoch in range(10):
# #             print(epoch)
#             x_adv = tf.convert_to_tensor(x_adv)
#             x_adv = tf.dtypes.cast(x_adv, tf.float32)
#             with tf.GradientTape(persistent=True) as g:
#                 g.watch(x_adv)
#     #             out = tf.argmax(m4.model(x_nat),axis=1)
#     #             out = tf.expand_dims(out, axis=1)
#     #             out = tf.dtypes.cast(out, tf.float64)
#     #             y = tf.dtypes.cast(y, tf.int64)
#     #             loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=out)
#                 out = m4.model(x_adv)
#                 y = tf.expand_dims(y, axis=1)
#                 out = tf.dtypes.cast(out, tf.float32)
#                 y = tf.dtypes.cast(y, tf.int64)
# #                 loss = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out)
#                 loss = [m4.defineLoss(k)(y,out) for k in np.arange(m4.params_dict['num_chunks'])]

#             grad = g.gradient(loss, x_adv)
#             grad = grad.eval(session=sess)
#             x_adv = x_adv.eval(session=sess)
#             grad = np.reshape(grad, [grad.shape[0], grad.shape[1]*grad.shape[2]*grad.shape[3]])
#             kxy, dxkxy = self.svgd_kernel(x_adv)
#             x_adv = np.reshape(x_adv, [x_adv.shape[0], x_adv.shape[1]*x_adv.shape[2]*x_adv.shape[3]])
#             x_adv += self.a * np.sign(c*(-(np.matmul(kxy, -grad) + dxkxy)/batch_size) + grad)
            
#             #we need to put back the dimensions since our model has a 4-dimensional input
#             x_adv = np.reshape(x_adv, [x_adv.shape[0], 28, 28, 1])
            
#             x_adv = np.clip(x_adv, x_nat - self.epsilon, x_nat + self.epsilon) 
            
#             x_adv = np.clip(x_adv, 0, 1) # ensure valid pixel range

#         return x_adv
    
#     def svgd_kernel(self, theta):
#         theta = np.reshape(theta, [theta.shape[0], theta.shape[1]*theta.shape[2]])
#         sq_dist = pdist(theta)
#         pairwise_dists = squareform(sq_dist)**2

#         h = np.median(pairwise_dists)  
#         h = np.sqrt(0.5 * h / np.log(theta.shape[0]))

#         # compute the rbf kernel
#         Kxy = np.exp( -pairwise_dists / h**2 / 2)

#         dxkxy = -np.matmul(Kxy, theta)
#         sumkxy = np.sum(Kxy, axis=1)
#         for i in range(theta.shape[1]):
#             dxkxy[:, i] = dxkxy[:,i] + np.multiply(theta[:,i],sumkxy)
#         dxkxy = dxkxy / (h**2)
#         return (Kxy, dxkxy)


# if __name__ == '__main__':
#     import json
#     import sys
#     import math
    
#     sess =  backend.get_session()

#     attack = LinfBLOBAttack(m4,
#                          epsilon=0.3,
#                          k=200,
#                          a=0.01,
#                          random_start=True)

#     num_eval_examples = 1000
#     eval_batch_size = 200
#     num_batches = int(math.ceil(num_eval_examples / eval_batch_size))

#     print('Iterating over {} batches'.format(num_batches))

#     #     x_adv_final = np.copy(mnist.test.images)

#     for restart in range(50):
#       # Initialize permutation
#         permutation = np.arange(num_eval_examples)
#         idx = np.arange(num_eval_examples)

        
#         (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
#         Y_test = np.squeeze(Y_test)
#         X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2],num_channels)
        
#         x_test = X_test 
#         y_test = Y_test
        
#         x_adv = x_test + np.random.uniform(-attack.epsilon, attack.epsilon, x_test.shape)
        

#       # per round
#         t0 = time.time()
#         for epoch in range(int(attack.k/10)):
#             print(epoch)
#             np.random.shuffle(idx)
#             x_test, x_adv, y_test = x_test[idx], x_adv[idx], y_test[idx]
#             permutation = permutation[idx]
            
#             for ibatch in range(num_batches):
#                 bstart = ibatch * eval_batch_size
#                 bend = min(bstart + eval_batch_size, num_eval_examples)
                
#                 x_batch = x_test[bstart:bend, :]
#                 x_batch_adv = x_adv[bstart:bend, :]
#                 y_batch = y_test[bstart:bend]
#             x_adv[bstart:bend, :] = attack.perturb(x_batch, x_batch_adv, y_batch, sess)
            

#         inv_permutation = np.argsort(permutation)
#         x_adv = x_adv[inv_permutation]
 
        
#         t1 = time.time()

#         print('restart: ', restart, '   time per batch: ', t1 - t0)

#         print('L2: ', np.mean(np.square(x_adv - x_test)))
#         print('Linf: ', np.max(np.abs(x_adv - x_test)))
#         x_adv = tf.convert_to_tensor(x_adv)
#         x_adv = tf.dtypes.cast(x_adv, tf.float32)
#         preds = np.argmax(sess.run(attack.model.predict(x_adv)), 1)
#         acc = np.mean(np.equal(preds, y_test))
#         print('adv acc: ', acc)